In [1]:
!nvidia-smi

Mon Aug 15 08:44:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

create voting classifier

In [2]:
from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
drive.mount('/content/gdrive')
!ls
%cd gdrive/My\ Drive/phd-thesis/
!ls

Mounted at /content/gdrive
gdrive	sample_data
/content/gdrive/My Drive/phd-thesis
 adasyn_xgboost_20_imb.ipynb
 adasyn_xgboost_20.ipynb
 adasyn_xgboost_fulldata_imb.ipynb
 adasyn_xgboost_fulldata.ipynb
 Attack_Types.csv
'bagging&ensemble.py'
 cnn.ipynb
 CNN-test-data
 CNN-test-data-balanced
 CNN-test-data-balanced-dropout
 CNN-test-data-imb
 createCNN_Balanced_fulldata.ipynb
 createCNN_Balanced.ipynb
 createNetwork-211025.py
 createNetwork-CNN-imb.ipynb
'createNetwork-testdata (1).ipynb'
 createNetwork-testdata.ipynb
 createNetwork-testdata.py
 ensemble_balance.ipynb
 ensemble_imb.ipynb
 Evaluate_adasyn_xgboost_model_fulldata.ipynb
 Evaluate_adasyn_xgboost_model_fulldata.ipynb（副本）
 Evaluate_adasyn_xgboost_model.ipynb
 evaluate_auc_3Algorithm_macro.ipynb
 evaluate_auc_3Algorithm_micro.ipynb
 Evaluate_CNN_Balacanced.ipynb
 Evaluate_CNN_imb.ipynb
 Evaluate_PR_3Algorithms.ipynb
 Evaluate_PR_CNN.ipynb
 Evaluate_PR_Voting.ipynb
 evaluate_PR_xgboost.ipynb
 evaluate_ros_balanced.ipynb
 Evaluat

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 30 14:40:43 2021
Comparison using Majority Voting classifier and ensemble methods
using somote for oversampling
nsl-kdd dataset is 20%
@author: Administrator
"""
import pandas as pd
import numpy as np
import sys
#bagging and ensemble methods
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, zero_one_loss, \
    accuracy_score, classification_report, roc_auc_score
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced

#other ensemble algorithm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from collections import Counter
import joblib

#traing dataset
#train_x = pd.read_csv('KDDTrain_x_20.csv',header=None)
#train_Y = pd.read_csv('KDDTrain_y_20.csv',header=None)
train_x = pd.read_csv('KDDTrain_x.csv',header=None)
train_Y = pd.read_csv('KDDTrain_y.csv',header=None)
train_x.drop(train_x.columns[[115,116,117]], axis=1, inplace=True)
train_Y[0]=list(train_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
print("train_x", train_x.head())
print("train_y", train_Y.head())
###

#testing dataset
#test_x = pd.read_csv('KDDTest_x_20.csv',header=None)
#test_Y = pd.read_csv('KDDTest_y_20.csv',header=None)
test_x = pd.read_csv('KDDTest_x.csv',header=None)
test_x.drop(test_x.columns[[115,116,117]], axis=1, inplace=True)

test_Y = pd.read_csv('KDDTest_y.csv',header=None)
test_Y[0]=list(test_Y[0].map({'benign':0, 'dos':1, 'probe':2, 'r2l':3, 'u2r':4}))
print("test_x", test_x.head())
print("test_y", test_Y.head())

train_Y = train_Y.values.ravel()
test_Y  = test_Y.values.ravel()
print('train dataset shape %s' % Counter(train_Y))
print('test dataset shape %s' % Counter(test_Y))
#print("before ADASYN \n", pd.Series(train_Y).value_counts())

######--------------- RandomUnderSampler-------------------------
mean_class_size = int(pd.Series(train_Y).value_counts().sum()/5)
print("mean_class_size", mean_class_size)

ratio= {0: mean_class_size, 1: mean_class_size}
        #'probe': mean_class_size,
        #'r2l': mean_class_size,
        #'u2r': mean_class_size
        #}

# check if ratio para is still used
# print("before \n", pd.Series(train_Y).value_counts())
# print('Original dataset shape %s' % Counter(train_Y))

from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=ratio, random_state=42)

train_x, train_Y = rus.fit_resample(train_x, train_Y)
print('RUS dataset shape %s' % Counter(train_Y))

"""
#***apply SMOTE
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
#***only smote
sm = SMOTE (sampling_strategy='auto', random_state=0)
train_x_sm, train_Y_sm = sm.fit_resample(train_x, train_Y)
print('smote dataset shape %s' % Counter(train_Y_sm))
"""
#*******try using ADASYN on train dataset to make balance
#only 
from imblearn.over_sampling import ADASYN
ad = ADASYN (sampling_strategy='not majority', random_state=0)
train_x_ad, train_Y_ad = ad.fit_resample(train_x, train_Y)
print('adasyn dataset shape %s' % Counter(train_Y_ad))

#***Tomek Links
from imblearn.under_sampling import TomekLinks
sampling_strategy = "all"
tl = TomekLinks(sampling_strategy=sampling_strategy)
# x_res, Y_res = tl.fit_resample(train_x_sm, train_Y_sm)
x_res, Y_res = tl.fit_resample(train_x_ad, train_Y_ad)
print('Tomek Links dataset shape %s' % Counter(Y_res))


#***smote+tomek
# smt = SMOTETomek(random_state=42)
# train_x_smt, train_Y_smt = smt.fit_resample(train_x, train_Y)
# print('after smote+tomek shape %s' % Counter(train_Y_smt))

# print("after SMOTE",  pd.Series(train_Y_smt).value_counts())
#sys.exit(0)
##-------------------------

#Try Majority Voting Method
def votingMethod():
    # log_clf = LogisticRegression(solver='lbfgs', max_iter=100)
    log_clf = LogisticRegression(solver='liblinear',max_iter=3000)
    # rnd_clf = RandomForestClassifier()
    svm_clf = SVC(probability=True,random_state=17)
    # nb_clf = GaussianNB()
    dt_clf = DecisionTreeClassifier(random_state=17)   #try random_state=1
    kn_clf = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
    
    voting_clf=VotingClassifier(
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('rf',rnd_clf), ('svm',svm_clf)],
        # voting='soft'
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('kn',kn_clf), ('svm',svm_clf)],
        #estimators=[('lc',log_clf),('dt',dt_clf), ('nv', nb_clf), ('kn', kn_clf), ('svm',svm_clf)],
        #estimators=[('lc',log_clf),('dt',dt_clf), ('kn', kn_clf), ('svm',svm_clf)],  #original  
        estimators=[('lc',log_clf), ('dt',dt_clf), ('kn', kn_clf), ('svm',svm_clf)],  #decision tree
        voting='soft'
    )
    
    #evaluation of voting classifier
    target_names = ['benign 0', 'dos 1', 'probe 2', 'r2l 3', 'u2r 4']
    #for clf in (log_clf, dt_clf, kn_clf, svm_clf, voting_clf):
    for clf in (log_clf, dt_clf, kn_clf, svm_clf, voting_clf): 
        #with and without smote
        #clf.fit(train_x, train_Y)   #without smote
        # clf.fit(train_x_sm, train_Y_sm)    #with SMOTE
        clf.fit(x_res,Y_res)    #with SMOTE+tomek
        pred_Y = clf.predict(test_x)
        #cross_val_score(clf, X_train, y2_train, cv=3, scoring="accuracy")
        #print(clf.__class__.__name__,accuracy_score(y2_test, y_pred))
        results = confusion_matrix(test_Y, pred_Y, labels= clf.classes_)
        error = zero_one_loss(test_Y, pred_Y) 
        print("validation:", clf.__class__.__name__,accuracy_score(test_Y, pred_Y))
        print("error:", error)
        print("result: \n" , results)
        
        joblib.dump(clf, "VotingClassifier.pkl")

        yy = clf.predict_proba(test_x)
        rac = roc_auc_score(test_Y, yy, multi_class='ovr', average ='macro')  #Y_res
        print("ROC AUC Score:", rac)
        print(classification_report(test_Y, pred_Y, target_names=target_names))   #Y_res
        crpt=geometric_mean_score(test_Y, pred_Y)
        print(classification_report_imbalanced(test_Y, pred_Y, target_names=target_names))  #Y_res
        print(f"The geometric mean is {crpt:.3f}") #Y_res
        
        
        
votingMethod()



######
"""
#try other ensembleMethods
def ensembleMethods():
    #ensemble methods
    dt_clf = DecisionTreeClassifier(random_state=17) 
    rnd_clf = RandomForestClassifier()
    ada_clf = AdaBoostClassifier(base_estimator=dt_clf,
                             n_estimators= 10,   #n_estimators= 500,
                             learning_rate=0.1,
                             random_state=17)
    
    gbc_clf = GradientBoostingClassifier(max_depth=None, n_estimators=10, learning_rate=0.1,random_state=17)
    hgbc_clf = HistGradientBoostingClassifier(max_depth=None, loss="categorical_crossentropy", learning_rate=0.1,random_state=17)
    etc_clf = ExtraTreesClassifier(max_depth=None, n_estimators=10, random_state=0)
    
    for clf in (dt_clf, rnd_clf, ada_clf, gbc_clf, hgbc_clf, etc_clf):  #test random forest algorithm
        clf.fit(train_x, train_Y)  #train model
        pred_Y = clf.predict(test_x)     #validation
        results = confusion_matrix(test_Y, pred_Y, labels= clf.classes_)
        print("validation:", clf.__class__.__name__,accuracy_score(test_Y, pred_Y))
        print(results)
        
ensembleMethods()
######
"""

train_x         0         1         2    3         4         5         6         7    \
0 -0.110249 -0.007679 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
1 -0.110249 -0.007737 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
2 -0.110249 -0.007762 -0.004919    0 -0.089486 -0.007736 -0.095076 -0.027023   
3 -0.110249 -0.007723 -0.002891    0 -0.089486 -0.007736 -0.095076 -0.027023   
4 -0.110249 -0.007728 -0.004814    0 -0.089486 -0.007736 -0.095076 -0.027023   

   8         9    ...  105  106  107  108  109  110  111  112  113  114  
0    0 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
1    0 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
2    0 -0.011664  ...    0    0    0    0    0    0    0    1    0    0  
3    1 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  
4    1 -0.011664  ...    0    0    0    0    0    0    0    0    0    0  

[5 rows x 115 columns]
train_y    0
0  0
1  0
2  1
3  0
4  0
test_

KeyboardInterrupt: ignored